In [ ]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA

In [ ]:
generos = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'ScienceFiction', 'TVMovie', 'Thriller', 'War', 'Western']

def read_parquet(pasta):
    arquivos_parquet = glob.glob(os.path.join(pasta, '*.parquet'))
    lista_df = [pd.read_parquet(arquivo) for arquivo in arquivos_parquet]
    df = pd.concat(lista_df, ignore_index=True)
    return df

In [ ]:
df_parquet_total = read_parquet('./preprocess/movies')[generos]
label = 'Action'
features = generos.copy()
features.remove(label)

In [ ]:
rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(df_parquet[features], df_parquet[label])

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Seleção das melhores características
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)

generos_selecionados = selector.get_feature_names_out()
print("Os 5 gêneros mais relevantes são:", generos_selecionados)

In [ ]:
# Definir e treinar o modelo Random Forest
model = RandomForestClassifier(criterion='entropy',random_state=42)
model.fit(X, y)

# Obter a importância das features
importances = model.feature_importances_

# Criar um dataframe para visualizar as importâncias
importancia_df = pd.DataFrame({'Gênero': features, 'Importância': importances})

# Ordenar as importâncias do maior para o menor
importancia_ordenada = importancia_df.sort_values(by='Importância', ascending=False)

# Definir um threshold (por exemplo, selecionar importância acima de 0.05)
threshold = 0.06
generos_selecionados = importancia_ordenada[importancia_ordenada['Importância'] > threshold]